In [20]:
# import pandas module
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
import yaml

In [21]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.palettes import Turbo256
from bokeh.models import Span, CustomJS, HoverTool
from bokeh.models.widgets import Select
from bokeh.layouts import column, layout
from bokeh.transform import dodge

output_notebook()

Loading BokehJS ...

In [22]:
def get_config():
    with open('config.yaml', 'r') as stream:
        config = yaml.safe_load(stream)
    return config

In [23]:
def load_files(subject):
    """
    Function loads the preprocessed files for the given subject; the pre-processing can be found in 'preprocessing_mental_health'
    
    Parameters
        subject:            The subject's initial
    
    Returns
        mental_data:       The dataframe containting the information for the 6 weeeks period
        baseline_data:     The dataframe that is created for the vegan weeks
        vegan_data:        The dataframe that is created for the baseline weeks
    
    """

    # give the path where the needed files are located; with the function get_config
    config = get_config()
    mental_state_files_path = (config['mental_state_files_path'])

    # give the path where the processed files are located
    processed_files_path = mental_state_files_path + '/processed'

    # create the mental file
    mental_filename = processed_files_path + f'/mental_{subject}.csv'

    # create the baseline file
    baseline_filename = processed_files_path + f'/baseline_{subject}.csv'

    # create the vegan file
    vegan_filename = processed_files_path + f'/vegan_{subject}.csv'
    
    # create dataframes for the whole period (mental), baseline and vegan period
    mental_data = pd.read_csv(mental_filename, sep=',')
    baseline_data = pd.read_csv(baseline_filename, sep=',')
    vegan_data = pd.read_csv(vegan_filename, sep=',')
    
    # set 'date' column to datetime 
    mental_data['date'] = pd.to_datetime(mental_data['date'], format='%Y%m%d')
    baseline_data['date'] = pd.to_datetime(baseline_data['date'], format='%Y%m%d')
    vegan_data['date'] = pd.to_datetime(vegan_data['date'], format='%Y%m%d')
    
    return mental_data, baseline_data, vegan_data

In [35]:
def line_plot(subject, mental_data):
    """
    function returns a line plot for the whole 6 weeks including baseline and vegan period
    
    Parameters
        subject:            The subject's initial
        mental_data:        The dataframe with the whole 6 weeks of mental health data
    
    Returns
        line plot for the given subject for over 6 weeks with a separation line at the point where
        the diet was switched.
    
    """
    # give source for line plot
    source = ColumnDataSource(data=mental_data)
    # create separate dataframe with only numeric data which is to plot
    numeric_data = mental_data.drop(['current companion', 'current activity', 'location', 'notes', 'subject', 'usage'], axis=1)

    # create empty list where all columns are to be added, set index to 0
    options = []
    index = 0

    # loop through columns and add to list of column names
    for (columnName, columnData) in numeric_data.iteritems():
        if index != 0:
            options.append(columnName)
        index = index + 1 

    # create dropdown menu to select which information top plot
    Axesselect = Select(title="Choose mood:", value="relaxed", options=options)
    Axesselect.js_on_change('value', CustomJS(args=dict(source=source, Axesselect=Axesselect), code="""
        source.data['relaxed'] = source.data[Axesselect.value];
        source.change.emit();
    """))

    controls = [Axesselect]
    inputs = column(*controls, width=300)

    # Create a plot
    p = figure(title=f'Subject {subject}:', plot_width=1000, plot_height=400, y_range=(0,7) , 
                x_axis_type='datetime', x_axis_label='Date', y_axis_label='Mood score')
    p.line('date', 'relaxed', source=source, color='#85D0FF')

    # add tooltips to the plot to show the exact date and scores; as well as the textual data which cannot be plotted to
    # provive additional information.
    tooltips=[('date','$x'),
                ('Score', '$y'),
                ('Notes', '@notes'),
                ('Current companion', '@{current companion}'),
                ('Usage', '@{usage}'),
                ('Location', '@location')]


    # add the tooltips to the plot
    p.add_tools(HoverTool(tooltips=tooltips))

    # set line at the border of baseline/vegan period
    border = time.mktime(datetime(2020, 10, 26, 3, 0, 0).timetuple())*1000
    daylight_savings_start = Span(location=border, dimension='height', line_color='#AAFF99', line_width=3)
    p.add_layout(daylight_savings_start)

    # finish off by formatting the dropdown and plot
    final_layout = layout([[inputs, p]])
    show(final_layout)

In [25]:
def bar_plot(subject, baseline_data, vegan_data):
    """
    function returns a line plot for the whole 6 weeks including baseline and vegan period
    
    Parameters
        subject:            The subject's initial
    
    Returns
        line plot for the given subject for over 6 weeks with a separation line at the point where
        the diet was switched.
    
    """

    baseline_mean = load_files(subject)[1].drop('date', axis=1).mean().round(2)
    vegan_mean = load_files(subject)[2].drop('date', axis=1).mean().round(2)
    
    baseline_mean = baseline_mean.to_frame().reset_index()
    vegan_mean = vegan_mean.to_frame().reset_index()

    vegan_baseline = pd.merge(vegan_mean, baseline_mean, on='index')
    vegan_baseline = vegan_baseline.rename(columns={'0_x': 'vegan', '0_y': 'baseline', 'index':'mood'})

    moods = vegan_baseline['mood'].unique().tolist()

    source = ColumnDataSource(vegan_baseline)

    p = figure(x_range=moods, y_range=(0, 7), title=f"Average moods baseline vs vegan for subject {subject}", plot_width=900)

    p.vbar(x=dodge('mood', -0.2, range=p.x_range), top='baseline', width=0.4, source=source, color="#8AE1FC", legend_label='Baseline')

    p.vbar(x=dodge('mood',  0.2,  range=p.x_range), top='vegan', width=0.4, source=source, color="#C3EB78", legend_label='Vegan')


    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.legend.location = "top_left"
    p.xaxis.major_label_orientation = "vertical"
    p.legend.click_policy= 'hide'

    show(p)

In [36]:
subject = 'a'
mental_data = load_files(subject)[0]
line_plot(subject, mental_data)